# Chapter 27: Web Performance

---

## Introduction

Speed is a feature. In the modern web, users expect pages to load instantly. Research consistently shows that even a one‑second delay can lead to:

- **Lower conversion rates** – Amazon found that every 100ms delay cost them 1% in sales.
- **Higher bounce rates** – 53% of mobile users abandon a site that takes longer than 3 seconds to load.
- **Poor user experience** – slow sites feel unresponsive and frustrate users.
- **Lower search rankings** – Google uses page speed as a ranking factor, especially for mobile searches.

Web performance is not just about making your site fast; it's about delivering a smooth, responsive experience that keeps users engaged. Performance optimization touches every part of your stack: HTML, CSS, JavaScript, images, and server configuration.

In this chapter, you will learn:

- **Why performance matters** – the impact on users, business, and SEO.
- **Core Web Vitals** – the key metrics Google uses to measure user experience.
- **Other important performance metrics** – how to measure and interpret them.
- **Optimizing HTML** – reducing DOM size, loading strategies, and resource hints.
- **Optimizing CSS** – critical CSS, minification, and avoiding expensive selectors.
- **Optimizing JavaScript** – minimizing main thread work, code splitting, and memory management.
- **Image optimization** – choosing formats, compression, lazy loading, and responsive images.
- **Caching strategies** – browser caching, service workers, and cache headers.
- **Performance testing tools** – Lighthouse, WebPageTest, and Chrome DevTools.

By the end of this chapter, you will have a toolkit of techniques to make your websites lightning fast.

---

## 27.1 Why Performance Matters

### User Experience Impact

Users perceive a fast site as more polished, trustworthy, and enjoyable. Performance directly affects:

- **Perceived performance** – how fast the user thinks the site is, which can be improved by loading critical content first and showing visual feedback.
- **Interactivity** – once the page is visible, how quickly it responds to user input.
- **Smoothness** – animations and scrolling should run at 60 frames per second.

### Business Impact

- **Conversion rates** – Walmart found that for every 1 second improvement in page load time, conversions increased by 2%.
- **Revenue** – faster sites keep users engaged longer, increasing page views and ad revenue.
- **Customer retention** – users are less likely to return to a slow site.

### SEO Impact

Since 2010, Google has used page speed as a ranking factor. In 2021, they introduced **Core Web Vitals** as part of their page experience ranking signals. Sites that meet these thresholds are rewarded with higher rankings, especially in mobile search.

---

## 27.2 Understanding Core Web Vitals

Core Web Vitals are a set of real‑world, user‑centered metrics that quantify key aspects of the user experience. They are part of Google's page experience signals.

### Largest Contentful Paint (LCP)

LCP measures **loading performance**: the time from when the page starts loading to when the largest content element (image, video, or block of text) is rendered within the viewport.

- **Good:** ≤ 2.5 seconds
- **Needs improvement:** 2.5 – 4.0 seconds
- **Poor:** > 4.0 seconds

**What affects LCP:**
- Slow server response times
- Render‑blocking JavaScript and CSS
- Slow resource load times (especially images)
- Client‑side rendering without optimizations

### First Input Delay (FID) / Interaction to Next Paint (INP)

FID measures **interactivity**: the time from when a user first interacts with a page (clicking a button, tapping a link) to the time when the browser is able to respond to that interaction. In 2024, FID is being replaced by **Interaction to Next Paint (INP)**, which considers all interactions, not just the first.

- **Good:** ≤ 100 ms (INP ≤ 200 ms)
- **Needs improvement:** 100 – 300 ms (INP 200 – 500 ms)
- **Poor:** > 300 ms (INP > 500 ms)

**What affects FID/INP:**
- Long tasks on the main thread (due to heavy JavaScript execution)
- Large JavaScript bundles
- Inefficient event handlers
- Third‑party scripts blocking the main thread

### Cumulative Layout Shift (CLS)

CLS measures **visual stability**: the sum total of all individual layout shifts that occur during the entire lifespan of the page. A layout shift happens when a visible element changes its position from one rendered frame to the next.

- **Good:** ≤ 0.1
- **Needs improvement:** 0.1 – 0.25
- **Poor:** > 0.25

**Common causes of CLS:**
- Images or videos without dimensions (no `width`/`height` attributes)
- Dynamically injected content (ads, embeds) without reserved space
- Web fonts causing FOIT/FOUT (Flash of Invisible/Unstyled Text)
- Actions waiting for network responses before updating DOM

**How to fix CLS:**
- Always set `width` and `height` attributes on images and videos (or use `aspect-ratio` in CSS).
- Reserve space for dynamic content (e.g., ads) with placeholders.
- Avoid inserting new content above existing content unless in response to user interaction.
- Use `font-display: optional` or `swap` to control font rendering.

---

## 27.3 Performance Metrics

Beyond Core Web Vitals, several other metrics help you understand performance.

### First Contentful Paint (FCP)

The time when the browser renders the first piece of content (text, image, SVG, etc.). It marks the start of the page loading from the user's perspective.

- **Good:** ≤ 1.8 seconds

### Time to Interactive (TTI)

The time when the page is fully interactive – it responds reliably to user input. TTI is measured by looking at the first time after FCP when the main thread has been idle for at least 5 seconds and all network requests are settled.

### Total Blocking Time (TBT)

The sum of all long tasks (tasks that take >50ms) between FCP and TTI. TBT correlates with FID/INP.

- **Good:** ≤ 200 ms

### Speed Index

A measure of how quickly content is visually displayed during page load. It calculates the average time at which visible parts of the page are displayed. Lower is better.

### Time to First Byte (TTFB)

The time between the browser requesting a page and receiving the first byte of the response. TTFB reflects server responsiveness and network latency.

- **Good:** ≤ 0.8 seconds

---

## 27.4 HTML Performance

HTML is the foundation of your page. Optimizing how it is delivered and parsed can significantly impact perceived performance.

### Minimizing DOM Size

A large DOM tree (many elements) increases memory usage, style calculations, and layout time. Aim for:

- Fewer than 1500 elements total.
- Depth less than 32 levels.
- No parent with more than 60 child elements.

**How to reduce DOM size:**
- Simplify your design – avoid unnecessary wrapper divs.
- Use semantic elements instead of generic containers.
- Dynamically render content only when needed (e.g., using JavaScript for off‑screen sections).

### Script Loading Strategies

The `<script>` tag can block HTML parsing. Use these attributes to control loading:

- **`async`** – Downloads the script during HTML parsing and executes as soon as it's available, potentially before parsing finishes. Order is not guaranteed.
- **`defer`** – Downloads the script during parsing but executes only after parsing is complete, in the order they appear.

```html
<!-- Loads and executes immediately, blocking parsing -->
<script src="critical.js"></script>

<!-- Downloads during parsing, executes after parsing (ideal for most scripts) -->
<script defer src="analytics.js"></script>

<!-- Downloads during parsing, executes as soon as ready (may block parsing) -->
<script async src="ad.js"></script>
```

**Best practice:** Place non‑critical scripts at the end of the `<body>` with `defer` or `async`. Inline critical scripts directly in the `<head>` to avoid round trips.

### Preload, Prefetch, and Preconnect

Resource hints tell the browser about upcoming resources, allowing it to start loading them early.

- **`<link rel="preload">`** – Tells the browser to download a resource that is needed for the current page as soon as possible. Use for critical fonts, hero images, or scripts.

```html
<link rel="preload" href="font.woff2" as="font" type="font/woff2" crossorigin>
<link rel="preload" href="critical.css" as="style">
```

- **`<link rel="prefetch">`** – Hints that a resource might be needed for the next page; the browser may download it during idle time.

```html
<link rel="prefetch" href="next-page.css" as="style">
```

- **`<link rel="preconnect">`** – Tells the browser to establish a connection to an origin before an HTTP request is made. Useful for third‑party origins (analytics, CDNs).

```html
<link rel="preconnect" href="https://api.example.com">
```

- **`<link rel="dns-prefetch">`** – Resolves the DNS for a domain early. Less powerful than preconnect but has broader support.

```html
<link rel="dns-prefetch" href="https://fonts.googleapis.com">
```

### Critical CSS

Inline the CSS required to style the above‑the‑fold content directly in the `<head>`. This eliminates the round trip for external CSS and renders the page much faster. The rest of the CSS can be loaded asynchronously.

```html
<head>
  <style>
    /* Critical CSS for above‑the‑fold content */
    header { ... }
    .hero { ... }
  </style>
  <link rel="preload" href="full-styles.css" as="style" onload="this.onload=null;this.rel='stylesheet'">
  <noscript><link rel="stylesheet" href="full-styles.css"></noscript>
</head>
```

Tools like **Critical** (from the creators of Lighthouse) can automate generating critical CSS.

---

## 27.5 CSS Performance

CSS can block rendering and cause expensive layout calculations. Optimize it carefully.

### Minification

Remove unnecessary whitespace, comments, and characters. Tools like **cssnano** or **Clean‑CSS** can reduce file size by 20‑30%.

```bash
# Example using cssnano
npm install cssnano-cli -g
cssnano input.css output.css
```

### Avoiding Expensive Selectors

Some selectors are more expensive for the browser to match. While modern browsers are fast, overly complex selectors can still impact performance, especially on large pages.

- Avoid universal selectors (`*`) in key paths.
- Keep selectors short and specific.
- Prefer classes over descendant selectors (`.nav a` is slower than `.nav-link`).
- Avoid nesting beyond three levels.

**Bad:**
```css
html body div.container ul li a.highlight span { ... }
```

**Good:**
```css
.highlight span { ... }
```

### Will‑Change Property

The `will-change` property hints to the browser that an element will change, allowing it to optimize ahead of time. Use sparingly, and only for properties that are likely to change (e.g., `transform`, `opacity`).

```css
.element {
    will-change: transform;
}
```

Overusing `will-change` can waste memory and degrade performance.

### Reduce Unused CSS

Tools like **PurgeCSS** (often used with Tailwind) can remove CSS that isn't used in your HTML/JavaScript. This drastically reduces file size.

```bash
# Example with PurgeCSS
purgecss --css styles.css --content index.html --output purified.css
```

### Asynchronous CSS Loading

For non‑critical CSS, load it asynchronously using the `media` trick or the `preload` approach shown earlier.

```html
<link rel="stylesheet" href="non-critical.css" media="print" onload="this.media='all'">
<noscript><link rel="stylesheet" href="non-critical.css"></noscript>
```

---

## 27.6 JavaScript Performance

JavaScript is often the biggest performance bottleneck because it can block rendering and consume the main thread.

### Minimizing Main Thread Work

- **Break up long tasks** – use `setTimeout`, `requestIdleCallback`, or `scheduler.postTask` to yield to the browser.
- **Use Web Workers** – for CPU‑intensive tasks, move them to a background thread.
- **Avoid forced synchronous layouts** – reading layout properties (like `offsetHeight`) after making style changes forces the browser to recalculate layout.

```javascript
// Bad: forces layout twice
element.style.width = '100px';
console.log(element.offsetHeight); // forces layout
element.style.height = '200px';

// Good: batch reads/writes
element.style.width = '100px';
element.style.height = '200px';
console.log(element.offsetHeight); // only one layout
```

### Debouncing and Throttling

As covered in Chapter 24, use debouncing/throttling for frequent events like `scroll`, `resize`, and `input` to avoid excessive function calls.

### Code Splitting

Instead of loading one giant JavaScript bundle, split your code into smaller chunks that are loaded on demand. Modern bundlers (Webpack, Vite, Rollup) support dynamic `import()`.

```javascript
// Instead of: import heavyLibrary from 'heavy'
// Use dynamic import
button.addEventListener('click', async () => {
    const heavyLibrary = await import('heavy');
    heavyLibrary.doSomething();
});
```

### Tree Shaking

Remove unused code from bundles. This requires using ES6 module syntax (`import`/`export`) and a bundler that supports tree shaking.

### Efficient DOM Manipulation

- Use **document fragments** to batch DOM insertions.
- Avoid accessing DOM properties in loops.
- Use `innerHTML` cautiously (can cause security issues and reflows).
- For many updates, consider using a virtual DOM library (like React) that batches updates.

### Memory Management

Memory leaks occur when the browser retains memory that is no longer needed. Common causes:

- **Unreferenced event listeners** – always remove listeners when the element is removed.
- **Detached DOM nodes** – if you remove an element but still hold a reference to it in JavaScript, it won't be garbage collected.
- **Closures** – be aware of variables captured in closures that could be kept alive longer than intended.

Use Chrome DevTools Memory tab to take heap snapshots and identify leaks.

---

## 27.7 Image Optimization

Images often account for the majority of a page's weight. Optimizing them yields huge gains.

### Choosing the Right Format

| Format | Use Case | Pros | Cons |
|--------|----------|------|------|
| **JPEG** | Photos, complex images | Small file size, wide support | No transparency, lossy |
| **PNG** | Logos, icons, screenshots | Lossless, transparency | Larger than JPEG |
| **GIF** | Simple animations | Simple, supported | Very large, limited colors |
| **SVG** | Icons, logos, illustrations | Scalable, small for vector | Not for photos |
| **WebP** | Modern replacement for JPEG/PNG | Smaller, supports transparency/animations | Not supported in very old browsers |
| **AVIF** | Next‑gen format | Even smaller than WebP, high quality | Limited support |

**Best practice:** Use WebP or AVIF with a fallback to JPEG/PNG via `<picture>`.

### Compression

- Use tools like **ImageOptim**, **TinyPNG**, or **Squoosh** to compress images without visible quality loss.
- For JPEGs, aim for quality 70‑85.
- For PNGs, reduce color depth if possible.

### Responsive Images with `srcset` and `sizes`

Deliver different image resolutions based on the user's screen size and device pixel ratio.

```html
<img srcset="small.jpg 300w,
             medium.jpg 600w,
             large.jpg 1200w"
     sizes="(max-width: 600px) 300px,
            (max-width: 1200px) 600px,
            1200px"
     src="fallback.jpg"
     alt="Description">
```

### Art Direction with `<picture>`

Use `<picture>` when you need different images for different viewport sizes (e.g., crop a wide image for mobile).

```html
<picture>
    <source media="(max-width: 600px)" srcset="mobile.jpg">
    <source media="(max-width: 1200px)" srcset="tablet.jpg">
    <img src="desktop.jpg" alt="Description">
</picture>
```

### Lazy Loading

Load images only when they are about to enter the viewport. Native lazy loading is supported in modern browsers:

```html
<img src="image.jpg" loading="lazy" alt="...">
```

For older browsers, use Intersection Observer (as shown in Chapter 24).

### Modern Image Delivery

Consider using an image CDN (like Cloudinary, Imgix) that automatically serves optimized images in the best format, with resizing and compression on the fly.

---

## 27.8 Caching Strategies

Caching stores copies of resources locally, reducing round trips and speeding up repeat visits.

### Browser Caching with HTTP Headers

Set appropriate `Cache-Control` headers on your server:

- `Cache-Control: max-age=31536000` – cache for one year (for versioned assets like `styles.[hash].css`).
- `Cache-Control: no-cache` – revalidate with the server each time (for HTML).
- `Cache-Control: public` – allows caching even if the response is normally non‑cacheable (like authenticated pages, if safe).
- `Cache-Control: private` – only cache in the browser, not in intermediate proxies.

Example for an Apache server (`.htaccess`):
```apache
<FilesMatch "\.(css|js|jpg|jpeg|png|gif|svg|woff2)$">
    Header set Cache-Control "max-age=31536000, public"
</FilesMatch>
```

### Service Workers

Service workers act as a programmable proxy between the browser and the network. They can cache assets and serve them offline, as well as implement advanced strategies like stale‑while‑revalidate.

A basic service worker registration:

```javascript
// In your main page
if ('serviceWorker' in navigator) {
    navigator.serviceWorker.register('/sw.js')
        .then(reg => console.log('SW registered:', reg))
        .catch(err => console.log('SW registration failed:', err));
}
```

Example service worker (`sw.js`):
```javascript
const CACHE_NAME = 'my-site-cache-v1';
const urlsToCache = [
    '/',
    '/styles/main.css',
    '/scripts/main.js',
    '/images/logo.png'
];

self.addEventListener('install', event => {
    event.waitUntil(
        caches.open(CACHE_NAME)
            .then(cache => cache.addAll(urlsToCache))
    );
});

self.addEventListener('fetch', event => {
    event.respondWith(
        caches.match(event.request)
            .then(response => response || fetch(event.request))
    );
});
```

Service workers enable offline support and can dramatically improve repeat visit performance.

### Cache Busting

When you update a file, you need to ensure browsers download the new version. Common technique: include a hash in the filename (e.g., `styles.a1b2c3.css`). When the file changes, the hash changes, forcing a new download. Build tools like Webpack do this automatically.

---

## 27.9 Performance Testing Tools

Measuring is the first step to optimization. Use these tools to identify bottlenecks.

### Lighthouse

Lighthouse is an open‑source tool built into Chrome DevTools. It audits performance, accessibility, best practices, and SEO, and provides actionable recommendations.

**How to run:**
- Open DevTools → Lighthouse tab → Generate report.
- Or run from the command line: `npx lighthouse https://example.com --view`

Lighthouse gives scores for Performance, Accessibility, Best Practices, SEO, and PWA. Under Performance, it shows metrics like FCP, LCP, TTI, TBT, and CLS.

### Chrome DevTools Performance Panel

The Performance panel allows you to record a profile of your page's runtime. It shows:

- Frames per second (FPS)
- CPU activity
- Network requests
- Layout/recalc events
- Long tasks

**How to use:**
1. Open DevTools → Performance tab.
2. Click the record button, interact with the page, then stop.
3. Analyze the flame chart to find expensive functions, forced layouts, and long tasks.

### WebPageTest

WebPageTest (webpagetest.org) is a powerful tool that tests your site from multiple locations and browsers. It provides:

- Filmstrip view of how the page loads visually.
- Waterfall chart of all requests.
- Core Web Vitals.
- Opportunities for improvement.

### PageSpeed Insights

PageSpeed Insights (developers.google.com/speed/pagespeed/insights) combines Lighthouse data with real‑world field data from the Chrome User Experience Report (CrUX). It gives both lab data (simulated) and field data (real users).

### Performance Budgets

Set performance budgets to prevent regressions. For example:

- JavaScript bundle size < 200 KB.
- LCP < 2.5 seconds.
- TBT < 200 ms.

Tools like **Lighthouse CI**, **Webpack Bundle Analyzer**, and **bundlesize** can enforce budgets in your CI/CD pipeline.

---

## Chapter Summary

In this chapter, you learned the essential techniques for optimizing web performance:

- **Why performance matters** – for users, business, and SEO.
- **Core Web Vitals** – LCP, FID/INP, and CLS, and how to improve them.
- **Key metrics** – FCP, TTI, TBT, Speed Index, TTFB.
- **HTML optimizations** – minimizing DOM size, script loading strategies, resource hints, critical CSS.
- **CSS optimizations** – minification, selector efficiency, `will-change`, and reducing unused CSS.
- **JavaScript optimizations** – main thread management, code splitting, efficient DOM manipulation, memory leaks.
- **Image optimization** – formats, compression, responsive images, lazy loading.
- **Caching strategies** – HTTP caching, service workers, cache busting.
- **Testing tools** – Lighthouse, DevTools Performance, WebPageTest, PageSpeed Insights.

### Key Takeaways

- Performance is a feature that directly impacts user satisfaction and business outcomes.
- Core Web Vitals are the key metrics to monitor and optimize.
- Optimize the critical rendering path: deliver essential CSS/JS early, defer the rest.
- Images are often the heaviest assets; optimize them ruthlessly.
- Use caching to avoid unnecessary network requests.
- Measure continuously; set budgets to prevent regressions.
- Performance is an ongoing process, not a one‑time fix.

### Practice Exercises

1. Run a Lighthouse audit on a website you've built. Note the performance score and opportunities. Implement at least three recommendations and re‑audit.

2. Use Chrome DevTools Performance panel to record loading of your site. Identify long tasks and forced layouts. Optimize them.

3. Implement responsive images on a page: create three versions of an image (small, medium, large) and use `srcset`/`sizes` to serve appropriate sizes. Verify with DevTools Network tab.

4. Set up a service worker to cache static assets for a simple site. Test offline functionality.

5. Use WebPageTest to analyze your site from a different location. Look at the filmstrip and waterfall. Identify the biggest bottlenecks.

6. Create a performance budget for a new project and integrate a tool like Lighthouse CI into your development workflow.

7. Experiment with different image formats: convert an image to WebP and AVIF. Compare file sizes and quality. Use the `<picture>` element to serve them with fallbacks.

---

## Coming Up Next

**Chapter 28: Web Security Fundamentals**

In the next chapter, you'll learn about the foundational security concepts every frontend developer must know: XSS, CSRF, secure coding practices, HTTPS, and security headers. You'll understand how to protect your users and your application from common attacks.